In [1]:
cd /media/datastorage/Phong/ucf101_v2/set3

/media/datastorage/Phong/ucf101_v2/set3


In [2]:
ls -l

total 16
drwxrwxr-x   2 bribeiro bribeiro 4096 fev  4 13:44 checkpoints/
drwxrwxr-x 103 bribeiro bribeiro 4096 fev  4 12:08 test/
drwxrwxr-x 103 bribeiro bribeiro 4096 fev  4 12:08 train/
-rw-rw-r--   1 bribeiro bribeiro  507 fev  4 13:44 ucf101_1_resnet152_set3.csv


In [3]:
# mkdir checkpoints

In [4]:
import pandas
import numpy as np
import os

d = {'id': [1, 2, 3, 4], 'pbest_value': [0, 0, 0, 0], 'pbest_file':['ucf101_1_resnet152_set3_best.hdf5',
                                                        'ucf101_2_resnet152_set3_best.hdf5',
                                                        'ucf101_3_resnet152_set3_best.hdf5',
                                                        'ucf101_4_resnet152_set3_best.hdf5'], 
                         'c_value': [0, 0, 0, 0], 'c_file':['ucf101_1_resnet152_set3.hdf5',
                                                             'ucf101_2_resnet152_set3.hdf5',
                                                             'ucf101_3_resnet152_set3.hdf5',
                                                             'ucf101_4_resnet152_set3.hdf5'], 
                         'pre_value': [0, 0, 0, 0], 'pre_file':['ucf101_1_resnet152_set3_pre.hdf5',
                                                             'ucf101_2_resnet152_set3_pre.hdf5',
                                                             'ucf101_3_resnet152_set3_pre.hdf5',
                                                             'ucf101_4_resnet152_set3_pre.hdf5'],
                         'training_flag':[0, 0, 0, 0]
    }
df = pandas.DataFrame(data=d)

In [5]:
#first instance only
df.to_csv(os.path.join('ucf101_1_resnet152_set3.csv'))

In [6]:
data_file = os.path.join('ucf101_1_resnet152_set3.csv')

def synch_read_data(data_file=''):
    while(True):
        try:
            df = pandas.read_csv(data_file, index_col=0)  
            break                     
        except:
            #waiting for 10s
            print("\n")
            for i in range(10,0,-1):
                print("re-read the file ....%2d" %i, end="\r", flush=True)
                time.sleep(1) 
    return df  

def synch_write_data(df,data_file=''):
    while(True):
        try:
            df.to_csv(data_file)  
            break                     
        except:
            #waiting for 10s
            print("\n")
            for i in range(10,0,-1):
                print("re-read the file ....%2d" %i, end="\r", flush=True)
                time.sleep(1) 
    return df  

def get_pbest_loc(row=0):
    df = synch_read_data(data_file)
    row=df.loc[row]
    pbest_value = row[1]
    file_name = row[2]
    return pbest_value, file_name

def set_pbest_loc(row, pbest_value):
    df = synch_read_data(data_file)
    df.loc[row, 'pbest_value'] = pbest_value
    synch_write_data(df,data_file)
    
def get_c_loc(row=0):
    df = synch_read_data(data_file)
    row=df.loc[row]
    c_value = row[3]
    file_name = row[4]
    return c_value, file_name

def set_c_loc(row, c_value):
    df = synch_read_data(data_file)
    df.loc[row, 'c_value'] = c_value
    synch_write_data(df,data_file)   

#    
def get_pre_loc(row=0):
    df = synch_read_data(data_file)
    row=df.loc[row]
    pre_value = row[5]
    file_name = row[6]
    return pre_value, file_name

def set_pre_loc(row, pre_value):
    df = synch_read_data(data_file)
    df.loc[row, 'pre_value'] = pre_value
    synch_write_data(df,data_file)
    
#training flag
def get_training_flag(row=0):
    df = synch_read_data(data_file)
    row=df.loc[row]
    training_flag = row[7]
    return training_flag

def set_training_flag(row, training_status):
    df = synch_read_data(data_file)
    df.loc[row, 'training_flag'] = training_status
    synch_write_data(df,data_file)

In [7]:
def find_distance(w1, w2):
    sqr_distance = 0
    
    w_np_1 = np.array(w1)
    w_fl_1 = w_np_1.flatten()
    w_np_2 = np.array(w2)
    w_fl_2 = w_np_2.flatten()
    
    for i in range(len(w_np_1)):
        x1_fl = w_fl_1[i].flatten()
        x2_fl = w_fl_2[i].flatten()

        tmp_dis = 0 
        for j in range(len(x1_fl)):
            tmp_dis = tmp_dis + (x1_fl[j]-x2_fl[j])**2

    #     print(tmp_dis)
        sqr_distance = sqr_distance + tmp_dis

    return sqr_distance**(1/2)  

In [8]:
from tensorflow.keras.callbacks import Callback
import pickle
import sys

#Stop training on val_acc
class EarlyStoppingByAccVal(Callback):
    def __init__(self, monitor='val_acc', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current >= self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping" % epoch)
            self.model.stop_training = True

#Save large model using pickle formate instead of h5            
class SaveCheckPoint(Callback):
    def __init__(self, model, dest_folder):
        super(Callback, self).__init__()
        self.model = model
        self.dest_folder = dest_folder
        
        #initiate
        self.best_val_acc = 0
        self.best_val_loss = sys.maxsize #get max value
          
    def on_epoch_end(self, epoch, logs={}):
        val_acc = logs['val_acc']
        val_loss = logs['val_loss']

        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            
            # Save weights in pickle format instead of h5
            print('\nSaving val_acc %f at %s' %(self.best_val_acc, self.dest_folder))
            weigh= self.model.get_weights()

            #now, use pickle to save your model weights, instead of .h5
            #for heavy model architectures, .h5 file is unsupported.
            fpkl= open(self.dest_folder, 'wb') #Python 3
            pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
            fpkl.close()
            
#             model.save('tmp.h5')
        elif val_acc == self.best_val_acc:
            if val_loss < self.best_val_loss:
                self.best_val_loss=val_loss
                
                # Save weights in pickle format instead of h5
                print('\nSaving val_acc %f at %s' %(self.best_val_acc, self.dest_folder))
                weigh= self.model.get_weights()

                #now, use pickle to save your model weights, instead of .h5
                #for heavy model architectures, .h5 file is unsupported.
                fpkl= open(self.dest_folder, 'wb') #Python 3
                pickle.dump(weigh, fpkl, protocol= pickle.HIGHEST_PROTOCOL)
                fpkl.close() 

In [9]:
from imutils import paths
from tqdm import tqdm
import pandas as pd 
import numpy as np
import shutil
import cv2
import os

In [ ]:
# # Open the .txt file which have names of training videos
# f = open("ucfTrainTestlist/trainlist01.txt", "r")
# temp = f.read()
# videos = temp.split('\n')

# # Create a dataframe having video names
# train = pd.DataFrame()
# train['video_name'] = videos
# train = train[:-1]
# train.head()

In [ ]:
# # Open the .txt file which have names of test videos
# with open("ucfTrainTestlist/testlist01.txt", "r") as f:
#     temp = f.read()
# videos = temp.split("\n")

# # Create a dataframe having video names
# test = pd.DataFrame()
# test["video_name"] = videos
# test = test[:-1]
# test.head()

In [ ]:
# def extract_tag(video_path):
#     return video_path.split("/")[0]

# def separate_video_name(video_name):
#     return video_name.split("/")[1]

# def rectify_video_name(video_name):
#     return video_name.split(" ")[0]

# # def move_videos(df, output_dir):
# #     if not os.path.exists(output_dir):
# #         os.mkdir(output_dir)
# #     for i in tqdm(range(df.shape[0])):
# #         videoFile = df['video_name'][i].split("/")[-1]
# #         videoPath = os.path.join("data", videoFile)
# #         shutil.copy2(videoPath, output_dir)
# #     print()
# #     print(f"Total videos: {len(os.listdir(output_dir))}")

In [ ]:
# train["tag"] = train["video_name"].apply(extract_tag)
# train["video_name"] = train["video_name"].apply(separate_video_name)
# train.head()

In [ ]:
# train["video_name"] = train["video_name"].apply(rectify_video_name)
# train.head()

In [ ]:
# test["tag"] = test["video_name"].apply(extract_tag)
# test["video_name"] = test["video_name"].apply(separate_video_name)
# test.head()

In [ ]:
# n = 101
# topNActs = train["tag"].value_counts().nlargest(n).reset_index()["index"].tolist()
# train_new = train[train["tag"].isin(topNActs)]
# test_new = test[test["tag"].isin(topNActs)]
# train_new.shape, test_new.shape

In [ ]:
# train_new = train_new.reset_index(drop=True)
# test_new = test_new.reset_index(drop=True)

In [ ]:
# def move_videos(df, output_dir):
#     if not os.path.exists(output_dir):
#         os.mkdir(output_dir)
#     for i in tqdm(range(df.shape[0])):
#         videoFile = df['video_name'][i].split("/")[-1]
#         videoTag = df['tag'][i]
#         videoPath = os.path.join("data", videoFile)
#         output_folder = os.path.join(output_dir, videoTag)
#         if not os.path.exists(output_folder):
#             os.mkdir(output_folder)
#         shutil.copy2(videoPath, output_folder)
#     print()
#     print(f"Total videos: {len(os.listdir(output_dir))}")

In [ ]:
# move_videos(train_new, "train")
# move_videos(test_new, "test")

In [10]:
"""
VideoFrameGenerator - Simple Generator
--------------------------------------
A simple frame generator that takes distributed frames from
videos. It is useful for videos that are scaled from frame 0 to end
and that have no noise frames.
"""

import glob
import logging
import os
import re
from math import floor
from typing import Iterable, Optional

import cv2 as cv
import numpy as np
from tensorflow.keras.preprocessing.image import (ImageDataGenerator,
                                                  img_to_array)
from tensorflow.keras.utils import Sequence

# from tensorflow.keras import backend as K
# # Don't Show Warning Messages
# import warnings
# warnings.filterwarnings('ignore')

# import gc; gc.enable()

log = logging.getLogger()




class VideoFrameGenerator(Sequence):  # pylint: disable=too-many-instance-attributes
    """
    Create a generator that return batches of frames from video
    # - rescale: float fraction to rescale pixel data (commonly 1/255.)
    - nb_frames: int, number of frames to return for each sequence
    - classes: list of str, classes to infer
    - batch_size: int, batch size for each loop
    - use_frame_cache: bool, use frame cache (may take a lot of memory for \
        large dataset)
    - shape: tuple, target size of the frames
    - shuffle: bool, randomize files
    - transformation: ImageDataGenerator with transformations
    - split: float, factor to split files and validation
    - nb_channel: int, 1 or 3, to get grayscaled or RGB images
    - glob_pattern: string, directory path with '{classname}' inside that \
        will be replaced by one of the class list
    - use_header: bool, default to True to use video header to read the \
        frame count if possible
    - seed: int, default to None, keep the seed value for split
    You may use the "classes" property to retrieve the class list afterward.
    The generator has that properties initialized:
    - classes_count: number of classes that the generator manages
    - files_count: number of video that the generator can provides
    - classes: the given class list
    - files: the full file list that the generator will use, this \
        is usefull if you want to remove some files that should not be \
        used by the generator.
    """

    def __init__(  # pylint: disable=too-many-statements,too-many-locals,too-many-branches,too-many-arguments
        self,
        # rescale: float = 1 / 255.0,
        nb_frames: int = 5,
        classes: list = None,
        batch_size: int = 16,
        use_frame_cache: bool = False,
        target_shape: tuple = (224, 224),
        shuffle: bool = True,
        transformation: Optional[ImageDataGenerator] = None,
        split_test: float = None,
        split_val: float = None,
        nb_channel: int = 3,
        glob_pattern: str = "./videos/{classname}/*.avi",
        use_headers: bool = True,
        seed=None,
        **kwargs,
    ):

        self.glob_pattern = glob_pattern

        # should be only RGB or Grayscale
        assert nb_channel in (1, 3)

        if classes is None:
            classes = self._discover_classes()

        # we should have classes
        if len(classes) == 0:
            log.warn(
                "You didn't provide classes list or "
                "we were not able to discover them from "
                "your pattern.\n"
                "Please check if the path is OK, and if the glob "
                "pattern is correct.\n"
                "See https://docs.python.org/3/library/glob.html"
            )

        # shape size should be 2
        assert len(target_shape) == 2

        # split factor should be a propoer value
        if split_val is not None:
            assert 0.0 < split_val < 1.0

        if split_test is not None:
            assert 0.0 < split_test < 1.0

        self.use_video_header = use_headers

        # then we don't need None anymore
        split_val = split_val if split_val is not None else 0.0
        split_test = split_test if split_test is not None else 0.0

        # be sure that classes are well ordered
        classes.sort()

        # self.rescale = rescale
        self.classes = classes
        self.batch_size = batch_size
        self.nbframe = nb_frames
        self.shuffle = shuffle
        self.target_shape = target_shape
        self.nb_channel = nb_channel
        self.transformation = transformation
        self.use_frame_cache = use_frame_cache

        self._random_trans = []
        self.__frame_cache = {}
        self.files = []
        self.validation = []
        self.test = []

        _validation_data = kwargs.get("_validation_data", None)
        _test_data = kwargs.get("_test_data", None)
        np.random.seed(seed)

        if _validation_data is not None:
            # we only need to set files here
            self.files = _validation_data

        elif _test_data is not None:
            # we only need to set files here
            self.files = _test_data
        else:
            self.__split_from_vals(
                split_val, split_test, classes, shuffle, glob_pattern
            )

        # build indexes
        self.files_count = len(self.files)
        self.indexes = np.arange(self.files_count)
        self.classes_count = len(classes)

        # to initialize transformations and shuffle indices
        if "no_epoch_at_init" not in kwargs:
            self.on_epoch_end()

        kind = "train"
        if _validation_data is not None:
            kind = "validation"
        elif _test_data is not None:
            kind = "test"

        self._current = 0
        self._framecounters = {}
        print(
            "Total data: %d classes for %d files for %s"
            % (self.classes_count, self.files_count, kind)
        )

    def count_frames(self, cap, name, force_no_headers=False):
        """Count number of frame for video
        if it's not possible with headers"""
        if not force_no_headers and name in self._framecounters:
            return self._framecounters[name]

        total = cap.get(cv.CAP_PROP_FRAME_COUNT)

        if force_no_headers or total < 0:
            # headers not ok
            total = 0
            # TODO: we're unable to use CAP_PROP_POS_FRAME here
            # so we open a new capture to not change the
            # pointer position of "cap"
            capture = cv.VideoCapture(name)
            while True:
                grabbed, _ = capture.read()
                if not grabbed:
                    # rewind and stop
                    break
                total += 1

        # keep the result
        self._framecounters[name] = total

        return total

    def __split_from_vals(self, split_val, split_test, classes, shuffle, glob_pattern):
        """ Split validation and test set """

        if split_val == 0 or split_test == 0:
            # no splitting, do the simplest thing
            for cls in classes:
                self.files += glob.glob(glob_pattern.format(classname=cls))
            return

        # else, there is some split to do
        for cls in classes:
            files = glob.glob(glob_pattern.format(classname=cls))
            nbval = 0
            nbtest = 0
            info = []

            # generate validation and test indexes
            indexes = np.arange(len(files))

            if shuffle:
                np.random.shuffle(indexes)

            nbtrain = 0
            if 0.0 < split_val < 1.0:
                nbval = int(split_val * len(files))
                nbtrain = len(files) - nbval

                # get some sample for validation_data
                val = np.random.permutation(indexes)[:nbval]

                # remove validation from train
                indexes = np.array([i for i in indexes if i not in val])
                self.validation += [files[i] for i in val]
                info.append("validation count: %d" % nbval)

            if 0.0 < split_test < 1.0:
                nbtest = int(split_test * nbtrain)
                nbtrain = len(files) - nbval - nbtest

                # get some sample for test_data
                val_test = np.random.permutation(indexes)[:nbtest]

                # remove test from train
                indexes = np.array([i for i in indexes if i not in val_test])
                self.test += [files[i] for i in val_test]
                info.append("test count: %d" % nbtest)

            # and now, make the file list
            self.files += [files[i] for i in indexes]
            print("class %s, %s, train count: %d" % (cls, ", ".join(info), nbtrain))

    def _discover_classes(self):
        pattern = os.path.realpath(self.glob_pattern)
        pattern = re.escape(pattern)
        pattern = pattern.replace("\\{classname\\}", "(.*?)")
        pattern = pattern.replace("\\*", ".*")

        files = glob.glob(self.glob_pattern.replace("{classname}", "*"))
        classes = set()
        for filename in files:
            filename = os.path.realpath(filename)
            classname = re.findall(pattern, filename)[0]
            classes.add(classname)

        return list(classes)

    def next(self):
        """ Return next element"""
        elem = self[self._current]
        self._current += 1
        if self._current == len(self):
            self._current = 0
            self.on_epoch_end()

        return elem

    # def get_validation_generator(self):
    #     """ Return the validation generator if you've provided split factor """
    #     return self.__class__(
    #         nb_frames=self.nbframe,
    #         nb_channel=self.nb_channel,
    #         target_shape=self.target_shape,
    #         classes=self.classes,
    #         batch_size=self.batch_size,
    #         shuffle=self.shuffle,
    #         # rescale=self.rescale,
    #         glob_pattern=self.glob_pattern,
    #         use_headers=self.use_video_header,
    #         _validation_data=self.validation,
    #     )

    # def get_test_generator(self):
    #     """ Return the validation generator if you've provided split factor """
    #     return self.__class__(
    #         nb_frames=self.nbframe,
    #         nb_channel=self.nb_channel,
    #         target_shape=self.target_shape,
    #         classes=self.classes,
    #         batch_size=self.batch_size,
    #         shuffle=self.shuffle,
    #         # rescale=self.rescale,
    #         glob_pattern=self.glob_pattern,
    #         use_headers=self.use_video_header,
    #         _test_data=self.test,
    #     )

    def on_epoch_end(self):
        """ Called by Keras after each epoch """

        if self.transformation is not None:
            self._random_trans = []
            for _ in range(self.files_count):
                self._random_trans.append(
                    self.transformation.get_random_transform(self.target_shape)
                )

        if self.shuffle:
            np.random.shuffle(self.indexes)          

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()

    def __len__(self):
        return int(np.floor(self.files_count / self.batch_size))

    def __getitem__(self, index):
        classes = self.classes
        shape = self.target_shape
        nbframe = self.nbframe

        labels = []
        images = []
        
        indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]

        transformation = None

        for i in indexes:

            video = self.files[i]
            classname = self._get_classname(video)

            # create a label array and set 1 to the right column
            label = np.zeros(len(classes))
            col = classes.index(classname)
            label[col] = 1.0

#             if video not in self.__frame_cache:
#                 frames = self._get_frames(
#                     video, nbframe, shape, force_no_headers=not self.use_video_header
#                 )
#                 if frames is None:
#                     # avoid failure, nevermind that video...
#                     continue

#                 # add to cache
#                 if self.use_frame_cache:
#                     self.__frame_cache[video] = frames

#             else:
#                 frames = self.__frame_cache[video]
            frames = self._get_frames(
                    video, nbframe, shape, force_no_headers=not self.use_video_header
                )

            # apply transformation
            # if provided
            if self.transformation is not None:
                transformation = self._random_trans[i]
                frames = [
                    self.transformation.apply_transform(frame, transformation)
                    if transformation is not None
                    else frame
                    for frame in frames
                ]

            # add the sequence in batch
            images.append(frames)
            labels.append(label)

        return np.array(images), np.array(labels)

    def _get_classname(self, video: str) -> str:
        """ Find classname from video filename following the pattern """

        # work with real path
        video = os.path.realpath(video)
        pattern = os.path.realpath(self.glob_pattern)

        # remove special regexp chars
        pattern = re.escape(pattern)

        # get back "*" to make it ".*" in regexp
        pattern = pattern.replace("\\*", ".*")

        # use {classname} as a capture
        pattern = pattern.replace("\\{classname\\}", "(.*?)")

        # and find all occurence
        classname = re.findall(pattern, video)[0]
        return classname

    def _get_frames(
        self, video, nbframe, shape, force_no_headers=False
    ) -> Optional[Iterable]:
        cap = cv.VideoCapture(video)
        total_frames = self.count_frames(cap, video, force_no_headers)
        orig_total = total_frames

        # if total_frames % 2 != 0:
        #     total_frames += 1

        frame_step = floor(total_frames / (nbframe - 1))
        # print('frame step = ', frame_step)
        # TODO: fix that, a tiny video can have a frame_step that is
        # under 1
        frame_step = max(1, frame_step)
        frames = []
        frame_i = 0

        # while True:
        #     grabbed, frame = cap.read()
        #     if not grabbed:
        #         break

        #     # ifixit: increase frame index
        #     frame_i += 1
        for index in range(nbframe):
            # print('index=', index)
            frame_pos = index*(frame_step-1)
            # print('frame pos=', frame_pos)
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_pos)
            grabbed, frame = cap.read()
            if not grabbed:
                break

            frame_i = frame_pos
            # print('frame_i=',frame_i)
            self.__add_and_convert_frame(
                frame, frame_i, frames, orig_total, shape, frame_step
            )

            if len(frames) == nbframe:
                break

        cap.release()

        if not force_no_headers and len(frames) != nbframe:
            # There is a problem here
            # That means that frame count in header is wrong or broken,
            # so we need to force the full read of video to get the right
            # frame counter
            return self._get_frames(video, nbframe, shape, force_no_headers=True)

        if force_no_headers and len(frames) != nbframe:
            # and if we really couldn't find the real frame counter
            # so we return None. Sorry, nothing can be done...
            log.error(
                f"Frame count is not OK for video {video}, "
                f"{total_frames} total, {len(frames)} extracted"
            )
            return None

        return np.array(frames)

    def __add_and_convert_frame(  # pylint: disable=too-many-arguments
        self, frame, frame_i, frames, orig_total, shape, frame_step
    ):
        #frame_i += 1
        # if frame_i in (1, orig_total) or frame_i % frame_step == 0:
        # crop center
        frame = self.__crop_center_square(frame)
        # resize
        frame = cv.resize(frame, shape)

        # use RGB or Grayscale ?
        frame = (
            cv.cvtColor(frame, cv.COLOR_BGR2RGB)
            if self.nb_channel == 3
            else cv.cvtColor(frame, cv.COLOR_RGB2GRAY)
        )

        # to np
        frame = img_to_array(frame)# * self.rescale

        # keep frame
        # print('append frame at frame_i= ', frame_i)
        frames.append(frame)

    def __crop_center_square(
        self, frame
    ):
        y, x = frame.shape[0:2]
        min_dim = min(y, x)
        start_x = (x // 2) - (min_dim // 2)
        start_y = (y // 2) - (min_dim // 2)
        return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]       

In [11]:
import os
import glob
import tensorflow as tf
from tensorflow.keras.applications.resnet import preprocess_input
# from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
# from tensorflow.keras.applications.mobilenet import preprocess_input
# from tensorflow.keras.applications.densenet import preprocess_input


# from keras_video import VideoFrameGenerator
# use sub directories names as classes
classes = [i.split(os.path.sep)[1] for i in glob.glob('train/*')]
classes.sort()
print(len(classes))

# some global params
SIZE = (224, 224)
CHANNELS = 3
NBFRAME = 4 #5
BS = 8
#
INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (5, 112, 112, 3)
# pattern to get videos and classes
glob_train_pattern='train/{classname}/*'
glob_test_pattern='test/{classname}/*'
# for data augmentation
data_train_aug = tf.keras.preprocessing.image.ImageDataGenerator(
    zoom_range=.1,
    # horizontal_flip=True,
    rotation_range=8,
    width_shift_range=.2,
    height_shift_range=.2,
    preprocessing_function=preprocess_input,
    )

data_test_aug = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_input,
    )
# Create video frame generator
train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_train_pattern,
    nb_frames=NBFRAME,
    # split=.33, 
    shuffle=True,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=data_train_aug,
    use_frame_cache=True)

# Create video frame generator
test = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_test_pattern,
    nb_frames=NBFRAME,
    # split=.33, 
    shuffle=False,
    batch_size=BS,
    target_shape=SIZE,
    nb_channel=CHANNELS,
    transformation=data_test_aug,
    use_frame_cache=True)

101
Total data: 101 classes for 9624 files for train
Total data: 101 classes for 3696 files for train


In [12]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, \
    MaxPool2D, GlobalMaxPool2D
from tensorflow.keras.models import Model
# from tf.keras.applications.mobilenet import preprocess_input


def build_convnet(shape=(224, 224, 3)):
    f1_base = tf.keras.applications.ResNet152(weights='imagenet', include_top=False, input_shape=shape)
    # f1_base = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=shape)
    # f1_base = tf.keras.applications.MobileNet(weights='imagenet', include_top=False, input_shape=shape)
#     f1_base = tf.keras.applications.DenseNet201(weights='imagenet', include_top=False, input_shape=shape)  
    f1_x = f1_base.output

    # #frozen layers    
    # for layer in f1_base.layers:
    #     layer.trainable = False  

    f1_x = tf.keras.layers.GlobalAveragePooling2D()(f1_x)

    model_1 = Model(inputs=[f1_base.input],outputs=[f1_x])        

    return model_1

In [13]:
from tensorflow.keras.layers import TimeDistributed, GRU, Dense, Dropout, GaussianNoise

def adv_action_model(shape=(4, 224, 224, 3), nbout=3):
    # Create our convnet with (112, 112, 3) input shape
    convnet = build_convnet(shape[1:])

    # for layer in convnet.layers:
    #     print(layer.name, ': ', layer.trainable)   
    
    # then create our final model
    model = tf.keras.Sequential()
    # add the convnet with (5, 112, 112, 3) shape
    model.add(TimeDistributed(convnet, input_shape=shape))
    # here, you can also use GRU or LSTM
    model.add(GRU(2048))
    #Regularization with noise
    model.add(GaussianNoise(0.1))
    # and finally, we make a decision network
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.4))
    model.add(Dense(nbout, activation='softmax'))
    return model

In [ ]:
# INSHAPE=(NBFRAME,) + SIZE + (CHANNELS,) # (5, 112, 112, 3)

# print(INSHAPE)
# print(len(classes))
# model = adv_action_model(INSHAPE, len(classes))
# # optimizer = tf.keras.optimizers.Adam(0.001)
# optimizer = tf.keras.optimizers.SGD(0.01)

# model.compile(
#     optimizer,
#     'categorical_crossentropy',
#     metrics=['acc'],
#     run_eagerly=True
# )


# print(model.summary())

In [ ]:
# # from tensorflow.python.keras.utils.data_utils import Sequence

# EPOCHS=60
# # create a "chkp" directory before to run that
# # because ModelCheckpoint will write models inside
# callbacks = [
#     # tf.keras.callbacks.ReduceLROnPlateau(verbose=1),
#     # tf.keras.callbacks.ModelCheckpoint(
#     #     'chkp/weights.{epoch:02d}-{val_loss:.2f}.hdf5',
#     #     verbose=1),
# ]
# model.fit(
#     train,
#     validation_data=test,
#     verbose=1,
    
#     epochs=EPOCHS,
# #     callbacks=callbacks,
#     # workers=2
# )

In [ ]:
# mkdir checkpoints

In [14]:
# Training
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam, SGD

import time, os
from math import ceil
import random

model_txt = 'st'
# Helper: Save the model.
savedfilename = os.path.join('checkpoints', 'ucf101_4_resnet152_set3.hdf5')
savedfilename_best = os.path.join('checkpoints', 'ucf101_4_resnet152_set3_best.hdf5')
savedfilename_pre = os.path.join('checkpoints', 'ucf101_4_resnet152_set3_pre.hdf5')

checkpointer = ModelCheckpoint(savedfilename,
                          monitor='val_accuracy', verbose=1, 
                          save_best_only=False, mode='max',save_weights_only=True)########

# Helper: TensorBoard
tb = TensorBoard(log_dir=os.path.join('svhn_output', 'logs', model_txt))

# Helper: Save results.
timestamp = time.time()
csv_logger = CSVLogger(os.path.join('svhn_output', 'logs', model_txt + '-' + 'training-' + \
    str(timestamp) + '.log'))

earlystopping = EarlyStoppingByAccVal(monitor='val_accuracy', value=0.9900, verbose=1)

def rand_scheduler(epoch, lr):
    rnd_lr = 10**(random.uniform(np.log10((1e-5)),np.log10((1e-1))))
#     if epoch < 30:
#         rnd_lr = 1e-2
#     else:    
#         rnd_lr = 1e-3
#     rnd_lr = lr
    print('random lr = ', rnd_lr)
    return rnd_lr

epochs = 20##!!!
lr = 1e-3
# decay = lr/epochs
# optimizer = Adam(lr=lr, decay=decay)
# optimizer = Adam(lr=lr)
optimizer = SGD(learning_rate=lr)

# train on multiple-gpus
# Create a MirroredStrategy.
strategy = tf.distribute.MirroredStrategy()
print("Number of GPUs: {}".format(strategy.num_replicas_in_sync))

# Open a strategy scope.
with strategy.scope():
    # Everything that creates variables should be under the strategy scope.
    # In general this is only model construction & `compile()`.
    model_mul = adv_action_model(INSHAPE, len(classes))
    model_mul.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
    # save initial model
    model_mul.save_weights(savedfilename)
    model_mul.save_weights(savedfilename_best)
    model_mul.save_weights(savedfilename_pre)
    
# step_size_train=ceil(train_set.n/train_set.batch_size)
# step_size_valid=ceil(valid_set.n/valid_set.batch_size)
# step_size_test=ceil(testing_set.n//testing_set.batch_size)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of GPUs: 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


2018-02-04 13:45:49.989330: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2018-02-04 13:45:50.583865: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10393 MB memory:  -> device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


In [15]:
from copy import deepcopy
import numpy as np
import keras
import math

#index of this pso
pso_index = 3

#number of neighbors (max=4)
num_neighbors = 4
#K coefficient
M = 1
u = 1

tmp_acc = 0
tmp_w = []
pbest_acc = 0
pbest_w = []

#accelerator coefficient
c1 = 0.5
c2 = 0.5
# w = 0.5

r1 = 0
r2 = 0

results_stack_accuracy = []
results_stack_val_accuracy = []
results_stack_loss = []
results_stack_val_loss = []

#threshold
# threshold = 0.97

# #iteration control
# i = 0
# iter_max = 40

warm_up = 0

#    
# time synchronize
number_of_pso = 4
training_start_flag = 1
training_finish_flag = 0

#set initial training flag to start
set_training_flag(pso_index, training_start_flag)

for index in range(warm_up, epochs): 
# while i < iter_max:
    #start training 
    set_training_flag(pso_index, training_start_flag)
    print(get_training_flag(pso_index))
    
    #save previous weight
    model_mul.save_weights(savedfilename_pre) 
    
    # result = model_mul.fit_generator(
    #     generator = train_set, 
    #     steps_per_epoch = step_size_train,
    #     validation_data = valid_set,
    #     validation_steps = step_size_valid,
    #     shuffle=True,
    #     epochs=1,
    #     callbacks=[checkpointer,tf.keras.callbacks.LearningRateScheduler(rand_scheduler)],
    # #     callbacks=[csv_logger, checkpointer, earlystopping],
    # #     callbacks=[tb, csv_logger, checkpointer, earlystopping],        
    #     verbose=1) 

    result = model_mul.fit(
        train,
        validation_data=test,
        verbose=1,   
        epochs=1,
        callbacks=[checkpointer,tf.keras.callbacks.LearningRateScheduler(rand_scheduler)],
        # workers=2
    ) 
    
    #save weights every iteration
#     model_mul.save_weights(savedfilename)
    
    tmp_acc = result.history.get('val_accuracy')[-1]
    tmp_w = model_mul.get_weights()
    tmp_lr = result.history.get('lr')[-1]
    
    #save current location in scoreboard
    set_c_loc(pso_index,tmp_acc) 
    
    if tmp_acc > pbest_acc:
        pbest_acc = tmp_acc
        pbest_w = tmp_w
        #save person best location
        set_pbest_loc(pso_index,pbest_acc)  
        # save best model
        model_mul.save_weights(savedfilename_best)        

    #set training flag to finish
    set_training_flag(pso_index, training_finish_flag)  
    print(get_training_flag(pso_index))
        
    # check if all PSOs is ready (flag==1)
    while(True):
        tmp_flag = 0
        for flg_i in range(number_of_pso):
            print("flg_i", flg_i, "flag", get_training_flag(flg_i))
            if(get_training_flag(flg_i) == 1):
                tmp_flag = 1
        if(tmp_flag==1):
            #waiting for 60s
            print("\n")
            for i in range(60,0,-1):
                print("waiting for ....%2d" %i, end="\r", flush=True)
                time.sleep(1)    
        else:
            print("end of waiting")
            break  
    
    r1 = random.uniform(0,1)
    r2 = random.uniform(0,1)
#     r3 = random.uniform(0,1)    
    
    #-----------nearest neighbor best--------------
    #get neighbor weights
    #1
    neighbor_c_acc_1, name_file_1 = get_c_loc(0)
    neighbor_c_acc_2, name_file_2 = get_c_loc(1)
    neighbor_c_acc_3, name_file_3 = get_c_loc(2)
    neighbor_c_acc_4, name_file_4 = get_c_loc(3)
    
    #get pre loc
    neighbor_pre_acc_1, name_pre_file_1 = get_pre_loc(0)
    neighbor_pre_acc_2, name_pre_file_2 = get_pre_loc(1)
    neighbor_pre_acc_3, name_pre_file_3 = get_pre_loc(2)
    neighbor_pre_acc_4, name_pre_file_4 = get_pre_loc(3)  
    
    #clone model for weights change
    model_clone = keras.models.clone_model(model_mul)
    
    model_clone.load_weights(os.path.join('checkpoints', name_file_1))
    neighbor_w_1 = model_clone.get_weights() 
    model_clone.load_weights(os.path.join('checkpoints', name_file_2))
    neighbor_w_2 = model_clone.get_weights()
    model_clone.load_weights(os.path.join('checkpoints', name_file_3))
    neighbor_w_3 = model_clone.get_weights()
    model_clone.load_weights(os.path.join('checkpoints', name_file_4))
    neighbor_w_4 = model_clone.get_weights()
    
    #clone model pre weights
    model_clone.load_weights(os.path.join('checkpoints', name_pre_file_1))
    neighbor_pre_w_1 = model_clone.get_weights()     
    model_clone.load_weights(os.path.join('checkpoints', name_pre_file_2))
    neighbor_pre_w_2 = model_clone.get_weights() 
    model_clone.load_weights(os.path.join('checkpoints', name_pre_file_3))
    neighbor_pre_w_3 = model_clone.get_weights()    
    model_clone.load_weights(os.path.join('checkpoints', name_pre_file_4))
    neighbor_pre_w_4 = model_clone.get_weights()  
    
    distance_1 = find_distance(neighbor_w_4,neighbor_w_1)
    distance_2 = find_distance(neighbor_w_4,neighbor_w_2)
    distance_3 = find_distance(neighbor_w_4,neighbor_w_3)
    
    #find the closest neighbor
    distances = list()
    distances.append((0,distance_1))
    distances.append((1,distance_2))
    distances.append((2,distance_3))
    distances.append((3,0))

    print('distances unsorted', distances)
    
    distances.sort(key=lambda tup: tup[1])
    print('distances ', distances)
    
    neighbors_idx = list()
    for i in range(num_neighbors):
        neighbors_idx.append(distances[i][0])        
    
    print('neighbors ids ', neighbors_idx)
    
    #get neighbor bests from the list
    neighbor_bests = list()
    #remove first element (self distance)
#     neighbors_idx.pop(0)
    
    for i in range(len(neighbors_idx)):
        neighbor_best_tmp, name_file_neighbor_best_tmp = get_pbest_loc(neighbors_idx[i])
        neighbor_bests.append((neighbors_idx[i],neighbor_best_tmp))
        print('neighbor_idx ', neighbors_idx[i])
        print('neighbor_best_tmp ', neighbor_best_tmp)

    # keep unsorted list of neighbor
    neighbor_tmp = deepcopy(neighbor_bests)       
    
    # sort the list for maximum accuracy   
    neighbor_bests.sort(key=lambda tup: tup[1], reverse=True)
    print('neighbor best ', neighbor_bests)
    #
    neighbor_best_value, name_file_neighbor_best = get_pbest_loc(neighbor_bests[0][0])
    print('name_file_neighbor_best ', name_file_neighbor_best)
    
    model_clone.load_weights(os.path.join('checkpoints', name_file_neighbor_best))
    neighbor_best_w = model_clone.get_weights()  
    #---------- end nearest neighbor best ----------
    
    #---------- cucker -----------------------------
    particle_w_i = neighbor_w_4
    sum_particle_tmp = 0
    
    #remove the fist (self)
    neighbor_tmp.pop(0)
    
    for j in range(len(neighbor_tmp)):
        if neighbor_tmp[j][0]==0:
            particle_w_j = neighbor_w_1
            particle_w_pre_j = neighbor_pre_w_1
            distance_ij = distance_1
            u = 0.2
        elif neighbor_tmp[j][0]==1:    
            particle_w_j = neighbor_w_2
            particle_w_pre_j = neighbor_pre_w_2
            distance_ij = distance_2
            u = 0.2
        elif neighbor_tmp[j][0]==2:    
            particle_w_j = neighbor_w_3
            particle_w_pre_j = neighbor_pre_w_3
            distance_ij = distance_3
            u = 0.2
            
        print('u ', u)
        print('distance_ij ', distance_ij)
        print('tmp_lr ', tmp_lr)
        #sum(K/(1+distance)*(particle_w_j-particle_w_i)
#         sum_particle_tmp -= M*u/(1+distance_ij)*(np.array(particle_w_pre_j)-np.array(particle_w_j))
        sum_particle_tmp =  sum_particle_tmp \
                            - M*u/(1+distance_ij)*(np.array(particle_w_pre_j)-np.array(particle_w_j)) \
                            + M*u*tmp_lr/(1+distance_ij)*np.array(particle_w_j)
        
    #---------- end cucker -------------------------

    #update networks' weights
    #     w = c1*r1*(np.array(pbest_w)-np.array(tmp_w))+c2*r2*(np.array(gbest_w)-np.array(tmp_w))
    #     w = r1*np.array(pbest_w)+r2*np.array(tmp_w)+r3*np.array(gbest_w)
    #     w = np.array(tmp_w)+tmp_lr*(c1*r1*(np.array(pbest_w)-np.array(tmp_w))+c2*r2*(np.array(gbest_w)-np.array(tmp_w)))
#     final_weight = np.array(tmp_w)+sum_particle_tmp+c2*r2*(np.array(neighbor_best_w)-np.array(tmp_w))
#     final_weight = np.array(tmp_w)+sum_particle_tmp

#     final_weight = np.array(tmp_w)+c2*r2*(np.array(neighbor_best_w)-np.array(tmp_w))
#     final_weight = np.array(tmp_w)+sum_particle_tmp+c1*r1*(np.array(pbest_w)-np.array(tmp_w))+c2*r2*(np.array(neighbor_best_w)-np.array(tmp_w))
    final_weight = np.array(tmp_w)+sum_particle_tmp+c2*r2*(np.array(neighbor_best_w)-np.array(tmp_w))
#     final_weight = np.array(tmp_w)+c2*r2*(np.array(neighbor_best_w)-np.array(tmp_w))
#     final_weight = np.array(tmp_w)+sum_particle_tmp
    
    model_mul.set_weights(final_weight)
    
    print('After ---> epoch=', index, ' r1=',r1, ' r2=',r2, ' current acc=', tmp_acc, ' local best=', pbest_acc, 
          ' neighbor index=', neighbor_bests[0][0], ' neighbor best=', neighbor_best_value)  
    
    results_stack_val_accuracy.append(result.history.get('val_accuracy')[-1])
    results_stack_accuracy.append(result.history.get('accuracy')[-1])
    results_stack_val_loss.append(result.history.get('val_loss')[-1])      
    results_stack_loss.append(result.history.get('loss')[-1])
    
#     i = i + 1
        
print(results_stack_val_accuracy)

1


2018-02-04 13:46:12.490383: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:776] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_2"
op: "FlatMapDataset"
input: "TensorDataset/_1"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: -2
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_flat_map_fn_44064"
    }
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\020FlatMapDataset:1"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
        dim {
          size: -1
        }
      }
      shape {
        dim {
          size: -1

random lr =  0.00024622340842443143


2018-02-04 13:47:11.053525: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
2018-02-04 13:47:11.335194: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2018-02-04 13:47:11.335637: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2018-02-04 13:47:11.335666: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2018-02-04 13:47:11.338032: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2018-02-04 13:47:11.338102: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


1203/1203 [==============================] - ETA: 0s - loss: 4.6023 - accuracy: 0.0200

...

distances unsorted [(0, 34287.81129446941), (1, 31313.812403628643), (2, 30613.36938465278), (3, 0)]
distances  [(3, 0), (2, 30613.36938465278), (1, 31313.812403628643), (0, 34287.81129446941)]
neighbors ids  [3, 2, 1, 0]
neighbor_idx  3
neighbor_best_tmp  0.8338744640350342
neighbor_idx  2
neighbor_best_tmp  0.8322510719299316
neighbor_idx  1
neighbor_best_tmp  0.8341450095176697
neighbor_idx  0
neighbor_best_tmp  0.8065476417541504
neighbor best  [(1, 0.8341450095176697), (3, 0.8338744640350342), (2, 0.8322510719299316), (0, 0.8065476417541504)]
name_file_neighbor_best  ucf101_2_resnet152_set3_best.hdf5
u  0.2
distance_ij  30613.36938465278
tmp_lr  3.5768746e-05


/tmp/ipykernel_28067/4153759187.py:237: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  - M*u/(1+distance_ij)*(np.array(particle_w_pre_j)-np.array(particle_w_j)) \
/tmp/ipykernel_28067/4153759187.py:238: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  + M*u*tmp_lr/(1+distance_ij)*np.array(particle_w_j)


u  0.2
distance_ij  31313.812403628643
tmp_lr  3.5768746e-05
u  0.2
distance_ij  34287.81129446941
tmp_lr  3.5768746e-05


/tmp/ipykernel_28067/4153759187.py:251: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  final_weight = np.array(tmp_w)+sum_particle_tmp+c2*r2*(np.array(neighbor_best_w)-np.array(tmp_w))


After ---> epoch= 19  r1= 0.5925205046385964  r2= 0.08188007799991781  current acc= 0.7602813839912415  local best= 0.8338744640350342  neighbor index= 1  neighbor best= 0.8341450095176697
[0.058170996606349945, 0.3660714328289032, 0.39475107192993164, 0.7670454382896423, 0.7610930800437927, 0.824404776096344, 0.8271104097366333, 0.8290043473243713, 0.8222402334213257, 0.5798160433769226, 0.7943723201751709, 0.7824675440788269, 0.8125, 0.823051929473877, 0.8254870176315308, 0.8338744640350342, 0.8238636255264282, 0.8295454382896423, 0.3298160135746002, 0.7602813839912415]


In [16]:
print(max(results_stack_val_accuracy))

0.8338744640350342


In [ ]:

# !wget --no-check-certificate https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
# !wget --no-check-certificate https://www.crcv.ucf.edu/data/UCF101/UCF101TrainTestSplits-RecognitionTask.zip

In [ ]:
# %%capture
# !unrar e UCF101.rar data/
# !unzip -qq UCF101TrainTestSplits-RecognitionTask.zip

In [ ]:
# move_videos(train_new, "train")
# move_videos(test_new, "test")

In [ ]:
ls -l

In [ ]:
# %mv UCF101.rar ucf101_v2
# %mv UCF101TrainTestSplits-RecognitionTask.zip ucf101_v2
# %mkdir ucf101_v2/set1
# %mv train ucf101_v2/set1
# %mv test ucf101_v2/set1
# %mv data ucf101_v2

In [ ]:
cd ../..

In [ ]:
ls -l